# Компьютерная семантика

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://www.nltk.org/howto/wordnet.html
* https://ruwordnet.ru/ru
* https://habr.com/ru/companies/unistar_digital/articles/687148/
* https://www.scaler.com/topics/nlp/wordnet-in-nlp/

## Задачи для совместного разбора

1\. Дано описание классов `Sence` и `Synset`. Создайте объект `Synset` на основе файлов из `data/rwn`.

In [4]:
from dataclasses import dataclass, field

@dataclass
class Sense:
    id: str
    name: str
    lemma: str
    main_word: str
    synt_type: str
    poses: str
    synset_id: str
    meaning: str

@dataclass
class Synset:
    id: str
    ruthes_name: str
    definition: str
    part_of_speech: str
    senses: list[Sense] = field(default_factory=list)
    hyponyms: list["Synset"] = field(default_factory=list, repr=False)
    hypernyms: list["Synset"] = field(default_factory=list, repr=False)

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
with open('sample_data/synsets.N.xml', 'r', encoding='utf8') as fp:
  syn = BeautifulSoup(fp)

In [ ]:
example_syn = syn.find('synset')
example_syn

<synset definition="" id="N12658" part_of_speech="N" ruthes_name="КОДИРОВАНИЕ ОТ ЗАВИСИМОСТИ">
<sense id="115643">МЕДИЦИНСКИЙ КОДИРОВАНИЕ</sense>
<sense id="115640">КОДИРОВАНИЕ</sense>
<sense id="115641">КОДИРОВАНИЕ ОТ ЗАВИСИМОСТЬ</sense>
<sense id="115642">КОДИРОВАНИЕ ЗАВИСИМОСТЬ</sense>
</synset>

In [ ]:
example_syn.attrs

{'id': 'N12658',
 'ruthes_name': 'КОДИРОВАНИЕ ОТ ЗАВИСИМОСТИ',
 'definition': '',
 'part_of_speech': 'N'}

In [ ]:
synset = Synset(**example_syn.attrs)
synset.id

'N12658'

In [ ]:
with open('sample_data/senses.N.xml', 'r', encoding='utf8') as fp:
  senses = BeautifulSoup(fp)

In [ ]:
senses_list = [
    Sense(**sense.attrs)
    for sense in senses.find_all('sense', synset_id=synset.id)
]
senses_list

[Sense(id='115643', name='МЕДИЦИНСКОЕ КОДИРОВАНИЕ', lemma='МЕДИЦИНСКИЙ КОДИРОВАНИЕ', main_word='КОДИРОВАНИЕ', synt_type='NG', poses='Adj N', synset_id='N12658', meaning='1'),
 Sense(id='115640', name='КОДИРОВАНИЕ', lemma='КОДИРОВАНИЕ', main_word='', synt_type='N', poses='', synset_id='N12658', meaning='3'),
 Sense(id='115641', name='КОДИРОВАНИЕ ОТ ЗАВИСИМОСТИ', lemma='КОДИРОВАНИЕ ОТ ЗАВИСИМОСТЬ', main_word='КОДИРОВАНИЕ', synt_type='NG', poses='N Prep N', synset_id='N12658', meaning='1'),
 Sense(id='115642', name='КОДИРОВАНИЕ ЗАВИСИМОСТИ', lemma='КОДИРОВАНИЕ ЗАВИСИМОСТЬ', main_word='КОДИРОВАНИЕ', synt_type='NG', poses='N N', synset_id='N12658', meaning='1')]

## Задачи для самостоятельного решения

In [1]:
!pip install Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.9/172.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.0 MB/s eta 0:00:00


In [2]:
from collections import defaultdict
from bs4 import BeautifulSoup
from Levenshtein import distance as lev
from nltk import word_tokenize

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

<p class="task" id="1"></p>

1\. На основе файлов `rwn/synsets.A.xml`, `rwn/synsets.N.xml` и `rwn/synsets.V.xml` создайте словарь `synsets`, где ключом является ID синсета, а значением - объекта класса `Synset`. Поля `senses`, `hyponyms`, `hypernyms` оставьте со значениями по умолчанию. Выведите количество объектов в полученном словаре на экран.

In [5]:
synsets = {}
synset_files = ['synsets.A.xml', 'synsets.N.xml', 'synsets.V.xml']
for file in synset_files:
  with open(f'sample_data/{file}', 'r', encoding='utf8') as fp:
    synset = BeautifulSoup(fp)
    syns = synset.find_all('synset')
    for syn in syns:
      s = Synset(**syn.attrs)
      synsets[s.id] = s

In [6]:
len(synsets)

49492

<p class="task" id="2"></p>

2\. Обновите поле `senses` у объектов в словаре `synsets` на основе файлов `rwn/senses.A.xml`, `rwn/senses.N.xml` и `rwn/senses.V.xml`. Выведите на экран среднее количество синонимов у синсетов.

In [34]:
senses_files = ['senses.A.xml', 'senses.N.xml', 'senses.V.xml']
cnt = 0
senses = []
for file in senses_files:
  with open(f'sample_data/{file}', 'r', encoding='utf8') as fp:
    senses.extend(BeautifulSoup(fp))

In [37]:
res = defaultdict(list)
for sen in senses:
  for s in sen.find_all('sense'):
    sense = Sense(**s.attrs)
    res[s['synset_id']].append(sense)

In [38]:
cnt = 0
for k, v in synsets.items():
  v.senses = res[k]
  cnt += len(res[k])

cnt / len(synsets)

2.6350925402085186

<p class="task" id="3"></p>

3\. Проанализируйте, какие типы отношений представлены в файле `rwn/relation.xml`. Выведите множество типов отношений на экран.
Обновите поля `hyponyms` и `hypernyms` у объектов в словаре `synsets` на основе файла `rwn/relation.xml`.

In [10]:
rel_files = ['synset_relations.A.xml', 'synset_relations.V.xml', 'synset_relations.N.xml']
relations = []
for file_ in rel_files:
  with open(f'sample_data/{file_}', 'r') as fp:
    relations.extend(BeautifulSoup(fp))

In [11]:
hypernyms = defaultdict(list)
hyponyms = defaultdict(list)
for rel in relations:
  for r in rel.find_all('relation'):
    if r['name'] == 'hypernym':
      hypernyms[r['parent_id']].append(synsets[r['child_id']])
    elif r['name'] == 'hyponym':
      hyponyms[r['parent_id']].append(synsets[r['child_id']])

In [12]:
for k, v in synsets.items():
  v.hypernyms = hypernyms[k]
  v.hyponyms = hyponyms[k]

<p class="task" id="4"></p>

4\. Напишите функцию `find_by_name`, которая ищет синсеты по вхождению заданного слова в поле `ruthes_name`. При поиске приводите введенное слово к нормальной форме и не учитывайте регистр слов. Функция должна вернуть список, отсортированный по возрастаю значений расстояния Левенштейна между названием синсета и введенным словом. Продемонстрируйте, какие синсеты находятся по слову "собака".

In [ ]:
!pip install pymorphy2

In [14]:
from pymorphy2 import MorphAnalyzer

In [27]:
def find_by_name(word, syns):
  res = []
  morph = MorphAnalyzer()
  word = morph.parse(word)[0].normal_form
  for k, v in syns.items():
    if word in v.ruthes_name.lower():
      res.append((syns[k], lev(word.lower(), v.ruthes_name.lower())))
  res = sorted(res, key=lambda x: x[1])
  return list(map(lambda x: x[0], res))

In [39]:
find_by_name('собака', synsets)

[Synset(id='A6565', ruthes_name='СОБАКА', definition='хищное млекопитающее семейства псовых', part_of_speech='Adj', senses=[Sense(id='4612', name='ПСИНЫЙ', lemma='ПСИНЫЙ', main_word='', synt_type='Adj', poses='', synset_id='A6565', meaning='1'), Sense(id='4946', name='ПЕСИЙ', lemma='ПЕСИЙ', main_word='', synt_type='Adj', poses='', synset_id='A6565', meaning='1'), Sense(id='5115', name='СОБАЧИЙ', lemma='СОБАЧИЙ', main_word='', synt_type='Adj', poses='', synset_id='A6565', meaning='1')]),
 Synset(id='N18450', ruthes_name='СОБАКА', definition='хищное млекопитающее семейства псовых', part_of_speech='N', senses=[Sense(id='5046', name='ПЕСИК', lemma='ПЕСИК', main_word='', synt_type='N', poses='', synset_id='N18450', meaning='1'), Sense(id='5113', name='ПЕС', lemma='ПЕС', main_word='', synt_type='N', poses='', synset_id='N18450', meaning='2'), Sense(id='5114', name='СОБАКА', lemma='СОБАКА', main_word='', synt_type='N', poses='', synset_id='N18450', meaning='1'), Sense(id='5116', name='СОБАЧКА

<p class="task" id="5"></p>

5\. Для пары слов "собака" и "кошка" найдите ближайший общий родительский синсет и выведите на экран его название. Синсет `A` назовем родительским синсетом синсета `B`, если от `B` можно подняться к `A` в таксономии синсетов, используя отношения гиперонимии. Найдите общий родительский синсет для пары слов "кошка" и "студент" и выведите на экран его название.

In [29]:
dog = find_by_name('собака', synsets)
cat = find_by_name('кошка', synsets)

In [30]:
def parse_hypernyms(hypernyms1, hypernyms2):
    for hyp1 in hypernyms1:
      for hyp2 in hypernyms2:
        if hyp1 == hyp2:
          return hyp1
    return parse_hypernyms(hypernyms1[0].hypernyms, hypernyms1[0].hypernyms)

In [40]:
parse_hypernyms(dog[0].hypernyms, cat[0].hypernyms)

Synset(id='A957', ruthes_name='ПОЗВОНОЧНОЕ ЖИВОТНОЕ', definition='', part_of_speech='Adj', senses=[Sense(id='122640', name='ПОЗВОНОЧНЫЕ', lemma='ПОЗВОНОЧНЫЙ', main_word='', synt_type='Adj', poses='', synset_id='A957', meaning='1')])

In [32]:
student = find_by_name('студент', synsets)

In [51]:
len(cat)

2

In [41]:
parse_hypernyms(cat[0].hypernyms, student[0].hypernyms)

Synset(id='A2250', ruthes_name='ЖИВОЙ ОРГАНИЗМ', definition='живое существо, как правило, низкоорганизованное', part_of_speech='Adj', senses=[Sense(id='24451', name='БИОЛОГИЧЕСКИЙ', lemma='БИОЛОГИЧЕСКИЙ', main_word='', synt_type='Adj', poses='', synset_id='A2250', meaning='3')])

<p class="task" id="6"></p>

6\. Для каждого слова из представленного текста найдите все возможные синонимы. Набор синонимов получите на основе поля `senses` у объектов `Synset`. В случае обнаружения точного совпадения введенного слова хотя бы с одним элементом из `senses`, весь набор `senses` трактуйте как синонимы. При поиске приводите слово к нормальной форме и не учитывайте регистр слов.

Составьте все возможные вариации данного предложения, заменяя слова на найденные синонимы. В случае отсутствия синонимов для какого-то из слов, используйте исходное слово для генерации предложения. Выведите на экран общее количество перефразированных предложений.

In [18]:
text = "Студент ужаснулся когда увидел задачу"

In [67]:
res = {}
morph = MorphAnalyzer()
for word in word_tokenize(text.lower()):
  res[morph.parse(word)[0].normal_form] = [sen.senses for sen in find_by_name(word, synsets)]

In [68]:
for k, v in res.items():
  names = []
  for lst in v:
    names.extend(list(map(lambda x: x.name.lower(), lst)))
  if k in names:
    res[k] = names
  else:
    res[k] = [k]
res

{'студент': ['студент',
  'студентка',
  'слушательский',
  'слушатель курсов',
  'слушательница',
  'слушательница курсов',
  'слушатель факультета',
  'слушатель'],
 'ужаснуться': ['леденящий ужас',
  'кровь стынет в жилах',
  'кровь леденеет в жилах',
  'кровь холодеет в жилах',
  'стынет кровь',
  'стынет кровь в жилах',
  'ужасаться',
  'приходить в ужас',
  'прийти в ужас',
  'ужаснуться',
  'приходить в ужас',
  'схватиться за голову',
  'ужаснуться',
  'прийти в ужас',
  'ужасаться',
  'хвататься за голову',
  'неприятно удивиться'],
 'когда': ['когда'],
 'увидеть': ['встреча',
  'иметь встречу',
  'видеть',
  'увидеть',
  'видаться',
  'видеться',
  'повидать',
  'встретить',
  'встречать',
  'свидеться',
  'увидеться',
  'пересечься',
  'встретиться',
  'встречаться',
  'пересекаться',
  'повидаться',
  'увидеть издали',
  'завидеть издали',
  'завидеть',
  'показ',
  'показывание',
  'показать',
  'показывать'],
 'задача': ['задачный',
  'задача',
  'максимальная задача',
  

In [72]:
import itertools

result_sents = []
for el in itertools.product(*res.values()):
    result_sents.append(' '.join(el))
print(f'Количество полученных предложений: {len(result_sents)}')

Количество полученных предложений: 31280


In [76]:
result_sents[::1000]

['студент леденящий ужас когда встреча задачный',
 'студент стынет кровь когда встречать задачный',
 'студент прийти в ужас когда увидеть издали задачный',
 'студент прийти в ужас когда иметь встречу задачный',
 'студентка леденящий ужас когда свидеться задачный',
 'студентка стынет кровь когда завидеть издали задачный',
 'студентка ужаснуться когда видеть задачный',
 'студентка прийти в ужас когда увидеться задачный',
 'слушательский леденящий ужас когда завидеть задачный',
 'слушательский стынет кровь в жилах когда увидеть задачный',
 'слушательский ужаснуться когда пересечься задачный',
 'слушательский прийти в ужас когда показ задачный',
 'слушатель курсов кровь стынет в жилах когда видаться задачный',
 'слушатель курсов стынет кровь в жилах когда встретиться задачный',
 'слушатель курсов ужаснуться когда показывание задачный',
 'слушатель курсов ужасаться когда видеться задачный',
 'слушательница кровь стынет в жилах когда встречаться задачный',
 'слушательница стынет кровь в жила

## Обратная связь
- [x] Хочу получить обратную связь по решению